In [48]:
!pip install firebase-admin --upgrade

Requirement already up-to-date: firebase-admin in /usr/local/envs/py3env/lib/python3.5/site-packages (4.5.3)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate('bgn-hack21-7017-firebase-adminsdk-i9cud-90b6e662d5.json')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

print('initialized')

initialized


In [3]:
# Load dataframe from csv
df = pd.read_csv("reed_uk.csv")
df.sample(10)

,category,city,company_name,geo,job_board,job_description,job_requirements,job_title,job_type,post_date,salary_offered,state
46788,science jobs,Cambridge,VRS,uk,reed,Apply now An exciting opportunity for an R&D ...,NaN,"R&D Scientist, Molecular Biology","Permanent, full-time",2/27/2018,"£24,000 - £30,000 per annum",Cambridgeshire
31889,marketing jobs,Northampton,We Are SSG,uk,reed,Apply now HTML Email Marketing ExecutiveLocat...,Required skills Data Dreamweaver Email Market...,HTML Email Marketing Executive,"Permanent, full-time",2/19/2018,Competitive salary,Northamptonshire
31578,law jobs,Basildon,Nelson Chambers,uk,reed,"Apply now My client is a award winning, Legal...",NaN,Private Client Solicitor,"Permanent, full-time",3/7/2018,"£30,000 - £45,000 per annum",Essex
43732,education jobs,Loughton,Capita Managed Services,uk,reed,Apply on recruiter's website School Crossing ...,"Required skills <P class=MsoNoSpacing style=""...",School Crossing Patrol - White Bridge Primary ...,"Permanent, full-time",2/27/2018,£7.88 per hour,Essex
49860,strategy consultancy jobs,Herefordshire,Parity Professionals,uk,reed,Apply now Project Manager - Contract - Herefo...,NaN,Project Manager,"Contract, full-time",3/2/2018,£380.00 - £400.00 per day,West Midlands (Region)
31727,customer service jobs,Peterborough,Taskmaster,uk,reed,Apply now Customer Care Consultant (Inbound C...,Required skills IT Account Manager Team leade...,Inbound Call Center IT First Line Support - Pe...,"Permanent, full-time",3/7/2018,"£18,000 per annum",Cambridgeshire
46418,charity jobs,London,Bestway,uk,reed,Apply on employer's website Bestway is a dyna...,NaN,Senior Channel Marketing Manager,"Permanent, full-time",2/23/2018,Salary negotiable,South East England
30642,law jobs,London,McGregor Boyall Associates Limited,uk,reed,Apply now Our client is in search of an exper...,NaN,Pensions/ M&A/ Bulk Annuities Lawyer,"Permanent, full-time",3/6/2018,Salary negotiable,South East England
31562,customer service jobs,Stratford-upon-Avon,Staffpoint,uk,reed,"Apply now My client, an award winning, rapidl...",Required skills Excellent customer service sk...,Part time Customer Service Advisor,"Permanent, part-time",2/27/2018,"£16,000 per annum",Warwickshire
36655,recruitment consultancy jobs,Central London,ESG Recruitment,uk,reed,Apply now Would you like to earn over Â£50k n...,Required skills Business Development Graduate...,TRAINEE RECRUITMENT CONSULTANT - EARN Â£50k+ N...,"Permanent, full-time",3/7/2018,"£20,000 - £50,000 per annum",London


In [4]:
df.describe()

,category,city,company_name,geo,job_board,job_description,job_requirements,job_title,job_type,post_date,salary_offered,state
count,50000,50000,50000,50000,50000,50000,20548,50000,50000,50000,50000,49980
unique,37,2918,5166,1,1,42057,14887,29155,9,66,7345,167
top,health jobs,London,Hays Specialist Recruitment Limited,uk,reed,Apply on employer's website Add an annual tur...,Required skills Recruitment,Administrator,"Permanent, full-time",3/7/2018,Salary negotiable,London
freq,1930,4349,1830,50000,50000,85,123,162,36864,8472,4539,5900


In [5]:
df.isnull().sum()

category                0
city                    0
company_name            0
geo                     0
job_board               0
job_description         0
job_requirements    29452
job_title               0
job_type                0
post_date               0
salary_offered          0
state                  20
dtype: int64

In [4]:
#Drop columns with null values
df.dropna(axis=1, inplace=True)
df.isnull().sum()

category           0
city               0
company_name       0
geo                0
job_board          0
job_description    0
job_title          0
job_type           0
post_date          0
salary_offered     0
dtype: int64

In [5]:
#Drop columsn where all values are identical
df.drop(["geo", "job_board"], axis=1, inplace=True)

#Drop city as that information can be gotten from state
#Drop post date as that doesn't affect the job itself
df.drop(["city", "post_date"], axis=1, inplace=True)
df.describe()

,category,company_name,job_description,job_title,job_type,salary_offered
count,50000,50000,50000,50000,50000,50000
unique,37,5166,42057,29155,9,7345
top,health jobs,Hays Specialist Recruitment Limited,Apply on employer's website Add an annual tur...,Administrator,"Permanent, full-time",Salary negotiable
freq,1930,1830,85,162,36864,4539


In [37]:
# Count unique values
df["category"].value_counts()

health jobs                           1930
education jobs                        1898
it jobs                               1890
logistics jobs                        1885
construction property jobs            1859
engineering jobs                      1858
accountancy jobs                      1856
sales jobs                            1855
accountancy qualified jobs            1836
social care jobs                      1823
admin secretarial pa jobs             1799
finance jobs                          1793
factory jobs                          1755
recruitment consultancy jobs          1747
motoring automotive jobs              1578
retail jobs                           1559
customer service jobs                 1548
catering jobs                         1536
law jobs                              1535
marketing jobs                        1463
other jobs                            1452
hr jobs                               1366
banking jobs                          1097
science job

In [6]:
# Edit values to be more descriptive
df["category"] = df["category"].str.replace(" jobs", "")
df.loc[df["category"] == "it", "category"] = "information technology"
df.loc[df["category"] == "graduate training internships", "category"] = "graduate internship"
df.loc[df["category"] == "fmcg", "category"] = "fast moving customer goods"
df.loc[df["category"] == "hr", "category"] = "human resources"

pd.unique(df["category"])

array(['catering', 'law', 'factory', 'retail', 'motoring automotive',
       'human resources', 'marketing', 'estate agent', 'customer service',
       'accountancy qualified', 'health', 'logistics', 'finance',
       'social care', 'recruitment consultancy', 'construction property',
       'engineering', 'admin secretarial pa', 'accountancy', 'sales',
       'information technology', 'education', 'other', 'security safety',
       'energy', 'apprenticeships', 'training', 'charity',
       'strategy consultancy', 'banking', 'purchasing',
       'fast moving customer goods', 'leisure tourism', 'science',
       'graduate internship', 'general insurance',
       'media digital creative'], dtype=object)

In [7]:
# Add an identifier column
df["job_id"] = df["company_name"] + " - " + df["job_title"]
df.head()

,category,company_name,job_description,job_title,job_type,salary_offered,job_id
0,catering,Haulfryn Group,Apply now New opportunity not to be missed! H...,Commis Chef,"Permanent, full-time","£18,323 per annum",Haulfryn Group - Commis Chef
1,law,Reed,Apply now This role requires an experienced s...,Corporate Legal Secretary,"Permanent, full-time","£18,000 - £22,000 per annum",Reed - Corporate Legal Secretary
2,factory,Swanstaff Recruitment Ltd,Apply now Job Title: Warehouse personLocation...,Warehouse Person,"Permanent, full-time",Salary not specified,Swanstaff Recruitment Ltd - Warehouse Person
3,law,Castles Solicitors,Apply now Castles Solicitors are looking for ...,Legal Secretary / Admin Assistant,"Permanent, full-time or part-time","£15,000 - £17,000 per annum, pro-rata",Castles Solicitors - Legal Secretary / Admin ...
4,factory,Marketplace Group,Apply now International Pharmaceutical Manufa...,Category Planner - Pharmaceutical,"Permanent, full-time","£37,000 - £45,000 per annum",Marketplace Group - Category Planner - Pharma...


In [11]:
df["job_type"].value_counts()

Permanent, full-time                 36864
Temporary, full-time                  5251
Contract, full-time                   3947
Permanent, part-time                  1361
Permanent, full-time or part-time      920
Temporary, part-time                   608
Temporary, full-time or part-time      506
Contract, full-time or part-time       290
Contract, part-time                    253
Name: job_type, dtype: int64

In [8]:
df["job_description"] = df["job_title"] + " " + df["job_type"] + " " + df["category"] + " " + df["job_description"]
df.head()

,category,company_name,job_description,job_title,job_type,salary_offered,job_id
0,catering,Haulfryn Group,"Commis Chef Permanent, full-time catering App...",Commis Chef,"Permanent, full-time","£18,323 per annum",Haulfryn Group - Commis Chef
1,law,Reed,"Corporate Legal Secretary Permanent, full-time...",Corporate Legal Secretary,"Permanent, full-time","£18,000 - £22,000 per annum",Reed - Corporate Legal Secretary
2,factory,Swanstaff Recruitment Ltd,"Warehouse Person Permanent, full-time factory ...",Warehouse Person,"Permanent, full-time",Salary not specified,Swanstaff Recruitment Ltd - Warehouse Person
3,law,Castles Solicitors,"Legal Secretary / Admin Assistant Permanent, f...",Legal Secretary / Admin Assistant,"Permanent, full-time or part-time","£15,000 - £17,000 per annum, pro-rata",Castles Solicitors - Legal Secretary / Admin ...
4,factory,Marketplace Group,"Category Planner - Pharmaceutical Permanent, f...",Category Planner - Pharmaceutical,"Permanent, full-time","£37,000 - £45,000 per annum",Marketplace Group - Category Planner - Pharma...


In [10]:
df.drop(["category", "job_title", "job_type", "salary_offered"], axis=1, inplace=True)
df.head()

,company_name,job_description,job_id
0,Haulfryn Group,"Commis Chef Permanent, full-time catering App...",Haulfryn Group - Commis Chef
1,Reed,"Corporate Legal Secretary Permanent, full-time...",Reed - Corporate Legal Secretary
2,Swanstaff Recruitment Ltd,"Warehouse Person Permanent, full-time factory ...",Swanstaff Recruitment Ltd - Warehouse Person
3,Castles Solicitors,"Legal Secretary / Admin Assistant Permanent, f...",Castles Solicitors - Legal Secretary / Admin ...
4,Marketplace Group,"Category Planner - Pharmaceutical Permanent, f...",Marketplace Group - Category Planner - Pharma...


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import pickle

tf = TfidfVectorizer()
jobdesc_tf = tf.fit_transform((df["job_description"]))
pickle.dump(jobdesc_tf, open("tfidf.pickle", "wb"))
print("done")

done


In [13]:
!gsutil cp 'tfidf.pickle' gs://bgn-hack21-7017.appspot.com/cos_sim_predict



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying file://tfidf.pickle [Content-Type=application/octet-stream]...
- [1 files][ 99.3 MiB/ 99.3 MiB]                                                
Operation completed over 1 objects/99.3 MiB.                                     


In [14]:
df.to_csv("joblistings.csv")

In [32]:
def get_recommendation(job_id, num_similar):
  
  job_index = df.loc[df["job_id"] == job_id].index[0]
  print(job_index)
  cos_sim = linear_kernel(jobdesc_tf[job_index:job_index+1], jobdesc_tf).flatten()
  
  output = list(cos_sim)
  top = sorted(range(len(output)), key=lambda i: output[i], reverse=True)[:num_similar+1]
  print(top)
  
  recommendation = pd.DataFrame(columns = ["job_id", "score"])
  count = 0
  
  for i in top:
    recommendation.at[count, "job_id"] = df["job_id"][i]
    recommendation.at[count, "score"] = output[i]
    count += 1

  return recommendation

i = df.iloc[109]["job_id"]
print(i)
get_recommendation(i, 10)

G2 Legal Limited  - Private Client Fee Earner
109
[109, 2570, 31237, 32357, 1642, 3086, 3493, 4504, 29992, 32453, 30406]


,job_id,score
0,G2 Legal Limited - Private Client Fee Earner,1
1,G2 Legal Limited - Private Client Fee Earner,1
2,Sacco Mann - Private Client Fee Earner,1
3,Douglas Scott Legal Recruitment - Property Fe...,0.770007
4,Sacco Mann - Commercial Property Fee Earner,0.711396
5,Bailey Hunter Ltd - Conveyancing Fee Earner,0.704463
6,G2 Legal Limited - Conveyancing Fee Earner,0.704463
7,RKRS Legal - Conveyancing Fee Earner,0.704463
8,Heat Recruitment - Conveyancing Fee Earner,0.704463
9,Douglas Scott Legal Recruitment - Conveyancin...,0.704463
